# Collapsed Gibbs sampler for the finite IBP approximation

In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
from copy import deepcopy

import VI_algorithms_lib
import valez_finite_VI_lib



## Draw data

In [2]:
np.random.seed(5365)

Num_samples = 500 # sample size
D = 2 # dimension
# so X will be a N\times D matrix

K_inf = 3 # take to be large for a good approximation to the IBP

alpha = 10 # IBP parameter

# Parameters to draw A from MVN
sigma_eps = .1 # variance of noise
sigma_A = 100

# generate data
Pi, Z, mu, A, X = valez_finite_VI_lib.generate_data(Num_samples, D, K_inf, sigma_A, sigma_eps, alpha)

K_approx = deepcopy(K_inf) # variational truncation


# Define collapsed Gibbs sampler

In [46]:
def likelihood_X(X, Z_Gibbs, sigma_eps, sigma_A): 
    
    assert np.shape(X)[0] == np.shape(Z_Gibbs)[0]
    
    D = np.shape(X)[1]
    N = np.shape(X)[0]
    K = np.shape(Z_Gibbs)[1]

    var = np.dot(Z_Gibbs.T, Z_Gibbs) + sigma_eps/sigma_A * np.eye(K)
    
    const = np.linalg.det(var)**(D/2)

    #const = (2*np.pi)**(N*D/2) * sigma_eps**((N-K)*D/2) * sigma_A**(K*D/2) * \
    #    np.linalg.det(var)**(D/2)
    
    mean_A = np.dot(np.linalg.solve(var, Z_Gibbs.T), X)
    
    likelihood = np.exp(-1/(2*sigma_eps) * np.trace(np.dot(X.T, X - np.dot(Z_Gibbs, mean_A)) ))
    
    return likelihood, mean_A

def draw_Znk(X, Z_Gibbs, sigma_eps, sigma_A, n,k): 
    
    D = np.shape(X)[1]
    N = np.shape(X)[0]
    K = np.shape(Z_Gibbs)[1]

    P_znk1 = (np.sum(Z_Gibbs[:,k]) - Z_Gibbs[n,k] + alpha/K)/(N - 1 + alpha)

    assert (P_znk1 > 0) & (P_znk1 < 1)
    
    P_znk0 = 1 - (np.sum(Z_Gibbs[:,k]) - Z_Gibbs[n,k] + alpha/K)/(N - 1 + alpha)
    
    Z_Gibbs[n,k] = 1
    [likelihood1, _] = likelihood_X(X, Z_Gibbs, sigma_eps, sigma_A)
    
    Z_Gibbs[n,k] = 0
    [likelihood0, _] = likelihood_X(X, Z_Gibbs, sigma_eps, sigma_A)
    
    P1 = likelihood1 * P_znk1# / (likelihood1 * P_znk1 + likelihood0 * P_znk0)
    print(likelihood1)
    Z_Gibbs[n,k] = np.random.binomial(1, P1)

## Initialize

In [47]:
Z_Gibbs = np.random.binomial(1, Pi, [ Num_samples, K_inf ])
[lik_X, mean_A] = likelihood_X(X, Z_Gibbs, sigma_eps, sigma_A) 

draw_Znk(X, Z_Gibbs, sigma_eps, sigma_A, 0,0)

0.0


In [8]:
A = np.random.normal(0,1,(3,3))
B = np.random.normal(0,1,(3,3))

C = np.linalg.solve(A,B)

print(np.dot(A,C))
print(B)


[[ 1.02746715 -0.99517561 -0.77697248]
 [ 0.71193284 -1.00608378 -1.60171903]
 [-0.25237748  0.2992343   0.85045211]]
[[ 1.02746715 -0.99517561 -0.77697248]
 [ 0.71193284 -1.00608378 -1.60171903]
 [-0.25237748  0.2992343   0.85045211]]
